<h1> Encrypted Inference-Linear Regression</h1>

Author:
<ul>
    <li>Hrishikesh Kamath - <a href="https://twitter.com/kamathhrishi">Twitter</a> - <a href="https://github.com/kamathhrishi">Github</a>
</ul>
Encrypted Inference is the process of performing inference with machine learning models such that model owner cannot observe the true input data nor can the data owner see the true model weights. The weights and data are encrypted by splitting them into shares and performiming computations according to a protocol. The general class of methods know as <b>Secure Multi Party Computation (SMPC)</b>. 

Below figure depicts MPC for ML models for 2 parties. 

<img height="600px" width="600px" src="Images/smpc_illustration.png"></img>

In this example we use Virtual Machine to demonstrate performing inference using SMPC. That is the workers are present on the same PC. If you want to understand how to perform remotely check out duet tutorials. 
In this example, we train a Linear regression model in plaintext on Boston Housing Dataset. Then we use the model for performing encrypted inference on test data. This tutorial uses protocol Falcon for 3 parties and SPDZ for 3 and 5 parties. 

In SyMPC the computation between parties occurs using a orchestrator which describes how computations should take place.

In [1]:
#running loss is nan
#batch size
#Define Linear regression model (figure out)
#criterion = torch.nn.MSELoss(reduction='mean') 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [2]:
#External libraries
import pandas as pd
import numpy as np
import time

In [3]:
#Import torch
import torch
import torch.nn as nn
import torch.utils.data as data_utils

In [4]:
#Set a manual seed to maintain consistency
torch.manual_seed(0)

<h2>Data Loading and Processing</h2>

In [5]:
!apt-get update
!apt-get install wget

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://security.debian.org/debian-security buster/updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.20.1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [6]:
#Download Boston housing dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

--2022-01-15 13:21:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49082 (48K) [application/x-httpd-php]
Saving to: ‘housing.data.6’

housing.data.6      100%[===================>]  47.93K   314KB/s    in 0.2s    

2022-01-15 13:21:45 (314 KB/s) - ‘housing.data.6’ saved [49082/49082]



In [7]:
#Import dataset and add headers
dataset=pd.read_csv("housing.data",delim_whitespace=True,
                    names=["crim","zn","indus",
                           "chas","nox","rm",
                           "age","dis","rad",
                           "tax","ptratio","black",
                           "lstat","medv"])

In [8]:
#Visualize and look at columns and rows of dataset
dataset.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [9]:
#Split data into features and target variables
features = dataset.drop("medv",axis=1)
targets = dataset["medv"]

In [10]:
#Normalize features
features = features.apply(
    lambda x: (x - x.mean()) / x.std()
)

In [11]:
features

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,-0.419367,0.284548,-1.286636,-0.272329,-0.144075,0.413263,-0.119895,0.140075,-0.981871,-0.665949,-1.457558,0.440616,-1.074499
1,-0.416927,-0.487240,-0.592794,-0.272329,-0.739530,0.194082,0.366803,0.556609,-0.867024,-0.986353,-0.302794,0.440616,-0.491953
2,-0.416929,-0.487240,-0.592794,-0.272329,-0.739530,1.281446,-0.265549,0.556609,-0.867024,-0.986353,-0.302794,0.396035,-1.207532
3,-0.416338,-0.487240,-1.305586,-0.272329,-0.834458,1.015298,-0.809088,1.076671,-0.752178,-1.105022,0.112920,0.415751,-1.360171
4,-0.412074,-0.487240,-1.305586,-0.272329,-0.834458,1.227362,-0.510674,1.076671,-0.752178,-1.105022,0.112920,0.440616,-1.025487
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.412820,-0.487240,0.115624,-0.272329,0.157968,0.438881,0.018654,-0.625178,-0.981871,-0.802418,1.175303,0.386834,-0.417734
502,-0.414839,-0.487240,0.115624,-0.272329,0.157968,-0.234316,0.288648,-0.715931,-0.981871,-0.802418,1.175303,0.440616,-0.500355
503,-0.413038,-0.487240,0.115624,-0.272329,0.157968,0.983986,0.796661,-0.772919,-0.981871,-0.802418,1.175303,0.440616,-0.982076
504,-0.407361,-0.487240,0.115624,-0.272329,0.157968,0.724955,0.736268,-0.667776,-0.981871,-0.802418,1.175303,0.402826,-0.864446


In [12]:
targets

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: medv, Length: 506, dtype: float64

In [13]:
#Convert features and targets into torch tensors
features = torch.tensor(features.values.astype(np.float32)) 
targets = torch.tensor(targets.values.astype(np.float32))

In [14]:
# Arguments for training
batch_size = 16
epochs = 300
train_test_split = 0.8
lr = 0.001

In [15]:
#Split dataset into train and test
train_indices=int(len(features)*train_test_split)

train_x = features[:train_indices]
train_y = targets[:train_indices]

test_x = features[train_indices+1:]
test_y = targets[train_indices+1:]

In [16]:
nom = int(input("Enter number of models to b created: "))
m=[]

Enter number of models to b created: 5


In [17]:
xtrain=[]
xtrain.append(np.array_split(train_x, nom))

ytrain=[]
ytrain.append(np.array_split(train_y, nom))

In [18]:
#Divide dataset into batches
def get_batches(X, y):
    batches = []
    for index in range(0,len(train_x)+1,batch_size):
        batches.append((X[index:index+batch_size],y[index:index+batch_size]))
    
    return batches

<h1>Plaintext Training</h1>

In [19]:
#Import syft
import syft as sy
sy.logger.remove()

In [20]:
#Define Linear regression model
class LinearSyNet(sy.Module):
    def __init__(self, torch_ref):
        super(LinearSyNet, self).__init__(torch_ref=torch_ref)
        self.fc1 = self.torch_ref.nn.Linear(13,1)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [21]:
"""#Define model, loss function and optimizer
for i in range(nom):
    model1 = LinearSyNet(torch)
    m.append(model1)
criterion = torch.nn.MSELoss(reduction='mean') 
optimizer = torch.optim.SGD(model.parameters(), lr=lr)"""

"#Define model, loss function and optimizer\nfor i in range(nom):\n    model1 = LinearSyNet(torch)\n    m.append(model1)\ncriterion = torch.nn.MSELoss(reduction='mean') \noptimizer = torch.optim.SGD(model.parameters(), lr=lr)"

In [22]:
xtest=[]
xtest.append(np.array_split(test_x, nom))

ytest=[]
ytest.append(np.array_split(test_y, nom))

In [23]:
print(len(train_x))
print(len(train_x[:404]))

404
404


In [24]:
#Training Loop
train_batches=get_batches(train_x,train_y)


for i in range(nom):

    model1 = LinearSyNet(torch)
    criterion = torch.nn.MSELoss(reduction='mean') 
    optimizer = torch.optim.SGD(model1.parameters(), lr=lr)
    print("model: ", i)
    for epoch in range(epochs):
      running_loss = 0.0
      for index in range(0,len(train_batches)):
        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
        optimizer.zero_grad()

        # get output from the model, given the inputs
        outputs = model1(train_batches[index][0]).reshape([-1])

        # get loss for the predicted output
        loss = criterion(outputs,train_batches[index][1])
        running_loss += loss
        # get gradients w.r.t to parameters
        loss.backward()

        # update parameters
        optimizer.step()

      test_accuracy = criterion(model1(test_x).reshape([-1]),test_y)
      if((epoch%50)==0):
         b =0
         print(f"Epoch {epoch}/{epochs}  Running Loss : {running_loss.item()/batch_size} and test loss: {test_accuracy.item()}")
            
    m.append(model1)

model:  0
Epoch 0/300  Running Loss : 978.0777587890625 and test loss: 367.94952392578125
Epoch 50/300  Running Loss : 52.76874923706055 and test loss: 108.14092254638672
Epoch 100/300  Running Loss : 43.0278434753418 and test loss: 47.97173309326172
Epoch 150/300  Running Loss : 41.08037567138672 and test loss: 30.05457305908203
Epoch 200/300  Running Loss : 40.12329864501953 and test loss: 23.080204010009766
Epoch 250/300  Running Loss : 39.579410552978516 and test loss: 20.645092010498047
model:  1
Epoch 0/300  Running Loss : 1006.2069091796875 and test loss: 355.1054992675781
Epoch 50/300  Running Loss : 53.09187698364258 and test loss: 110.3436050415039
Epoch 100/300  Running Loss : 43.0759391784668 and test loss: 48.66814422607422
Epoch 150/300  Running Loss : 41.09862518310547 and test loss: 30.31635856628418
Epoch 200/300  Running Loss : 40.13597869873047 and test loss: 23.175142288208008
Epoch 250/300  Running Loss : 39.59026336669922 and test loss: 20.673261642456055
model:  

In [25]:
"""#Training Loop
for i in range(nom):
    train_batches=get_batches(train_x,train_y)
    print("model: ", i)
    for epoch in range(epochs):
      running_loss = 0.0
      for index in range(0,len(train_batches)):
        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
        optimizer.zero_grad()

        # get output from the model, given the inputs
        outputs = m[i](train_batches[index][0]).reshape([-1])

        # get loss for the predicted output
        loss = criterion(outputs,train_batches[index][1])
        running_loss += loss
        # get gradients w.r.t to parameters
        loss.backward()

        # update parameters
        optimizer.step()

      test_accuracy = criterion(m[i](xtest[0][i]).reshape([-1]),ytest[0][i])#use xtest and ytest
      if((epoch%50)==0):
         print(f"Epoch {epoch}/{epochs}  Running Loss : {running_loss.item()/batch_size} and test loss: {test_accuracy.item()}")"""

'#Training Loop\nfor i in range(nom):\n    train_batches=get_batches(train_x,train_y)\n    print("model: ", i)\n    for epoch in range(epochs):\n      running_loss = 0.0\n      for index in range(0,len(train_batches)):\n        # Clear gradient buffers because we don\'t want any gradient from previous epoch to carry forward, dont want to cummulate gradients\n        optimizer.zero_grad()\n\n        # get output from the model, given the inputs\n        outputs = m[i](train_batches[index][0]).reshape([-1])\n\n        # get loss for the predicted output\n        loss = criterion(outputs,train_batches[index][1])\n        running_loss += loss\n        # get gradients w.r.t to parameters\n        loss.backward()\n\n        # update parameters\n        optimizer.step()\n\n      test_accuracy = criterion(m[i](xtest[0][i]).reshape([-1]),ytest[0][i])#use xtest and ytest\n      if((epoch%50)==0):\n         print(f"Epoch {epoch}/{epochs}  Running Loss : {running_loss.item()/batch_size} and test l

<h1>Encrypted Inference</h1>

In [26]:
#SyMPC imports required for encrypted inference
import sympc
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor
from sympc.protocol import Falcon,FSS

In [27]:
def get_clients(n_parties):
  #Generate required number of syft clients and return them.

  parties=[]
  for index in range(n_parties): 
      parties.append(sy.VirtualMachine(name = "worker"+str(index)).get_root_client())

  return parties

In [28]:
def split_send(data,session):
    """Splits data into number of chunks equal to number of parties and distributes it to respective 
       parties.
    """
    data_pointers = []
    
    split_size = int(len(data)/len(session.parties))+1
    for index in range(0,len(session.parties)):
        ptr=data[index*split_size:index*split_size+split_size].share(session=session)
        data_pointers.append(ptr)
        
    return data_pointers

In [29]:
def inference(n_clients,protocol=None):
    
    
  # Get VM clients 
  parties=get_clients(n_clients)

  # Setup the session for the computation
  if(protocol):
     session = Session(parties = parties,protocol = protocol)
  else:
     session = Session(parties = parties)
        
  SessionManager.setup_mpc(session)
  pointers=[]
  mpc_model=[]
  for i in range(nom):
        #Split data and send data to clients
        pointers.append(split_send(xtest[0][i],session))

        #Encrypt model 
        mpc_model.append(m[i].share(session))

        #Encrypt test data
        #test_data=MPCTensor(secret=test_x, session = session)
  for i in range(nom):
        #Perform inference and measure time taken
        start_time = time.time()

        results = []
        #print(len(pointers[1]))
        for j in range(len(pointers[0])):
            encrypted_results = mpc_model[i](pointers[i][j])
            plaintext_results = encrypted_results.reconstruct()
            results.append(plaintext_results)

        end_time = time.time()

        print(f"Time for inference: {end_time-start_time}s")

        predictions = torch.cat(results).reshape([-1])

        #Calculate Loss
        print("MSE Loss: ",criterion(predictions,ytest[0][i]).item())

        return predictions

In [30]:
predictions=inference(3,Falcon("semi-honest"))

Time for inference: 0.153364896774292s
MSE Loss:  53.92003631591797


We can see that the prediction values and mean squared error values are almost the same as final model. Small differences are due to precision loss.

In [31]:
for index in range(0,10):
    print(f"Index {index}")
    print(f"Encrypted Prediction Output {predictions[index].item()}")
    print(f"Plaintext Prediction Output {plaintext_predictions[index].item()}")
    print(f"Expected Prediction: {test_y[index]}")
    print("\n")

Index 0
Encrypted Prediction Output 2.636962890625


NameError: name 'plaintext_predictions' is not defined

<h1> Conclusion </h1>

Falcon can also provide a malicious security guarantee for an honest majority at the cost of higher inference time. Malicious security ensures that all the parties compute according to the protocol and do not deviate from protocol or tamper with shares. 

In [ ]:
predictions=inference(3,Falcon("malicious"))

When we do not pass any protocol to session, SyMPC uses SPDZ and FSS protocol with semi-honest security type. 

SPDZ is used for multiplication and related operations (convolution,matmul,etc).
Functional Secret Sharing (FSS) for other operations such as comparison, equality, maxpool, etc. 

FSS works only for 2 parties while SPDZ could extend to N parties. 

Linear regression uses only matmul which utilizes SPDZ protocol allowing us to run linear regression with several parties in this tutorial. 

In [ ]:
predictions=inference(3)

In [ ]:
predictions=inference(4)

In [ ]:
#predictions=inference(5)

<center><h3> Comparison </h3></center>

| Protocol | Security Type| Parties | Inference Time (s) |
| --- | --- | --- | --- |
| Plaintext | |  | 0.000534|
| Falcon | Semi-honest | 3 | 0.118 |
| Falcon | Malicious | 3 | 0.6659 |
| SPDZ| Semi-honest | 3 | 0.4993 |
| SPDZ | Semi-honest | 5 | 1.3192|

<b>Note:</b> The above table is only for comparison. The inference time varies for different PC specs and CPU load.



Falcon provides faster inference for a 3 parties setting. While, SPDZ allows inference for N number of parties. Both allow inference with almost same accuracy as plaintext. 

Although Falcon is much more faster, it isn't scalable because it is applicable for only 3 parties. Further, it is less secure. Since it uses 2-out-of-3 sharing where each party recieves two shares allowing 2 parties to reconstruct a secret without other party knowing. Falcon also assumes that majority of parties are honest (2 in this case). 

While, SPDZ and FSS distributes a single share to every party requiring shares from all parties for reconstruction ensuring no parties could collude. Currently SyMPC provides support for SPDZ and FSS with semi-honest security guarantee. This allows parties to tamper with shares leading to incorrect results. 

<h3>What's next?</h3>

SyMPC is still under development! We will add here more features as soon they are stable enough, stay tuned! 🕺

If you enjoyed this tutorial, show your support by Starring SyMPC! 🙏